## Hourly temperature for main cities in Spain

Information exported from ERA5
(https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=overview)

2m dewpoint temperature	(K)

This parameter is the temperature to which the air, at 2 metres above the surface of the Earth, would have to be cooled for saturation to occur. It is a measure of the humidity of the air. Combined with temperature and pressure, it can be used to calculate the relative humidity. 2m dew point temperature is calculated by interpolating between the lowest model level and the Earth's surface, taking account of the atmospheric conditions. This parameter has units of kelvin (K). Temperature measured in kelvin can be converted to degrees Celsius (°C) by subtracting 273.15.

UTC (Coordinated Universal Time)

In [61]:
#import libraries
import pandas as pd
import numpy as np
import cdsapi
import netCDF4

## List of cities
Main cities: Latitud, longitud 

- Asturias - Oviedo: North 43.4°, West -5.8°, South 43.3°, East -5.7°
- Madrid - Madrid: North 40.4°, West -3.7°, South 40.3°, East -3.6°
- Cataluña - Barcelona: North 41.4°, West 2.1°, South 41.3°, East 2.2°
- Comunidad Valenciana - Valencia: North 39.5°, West -0.37°, South 39.4°, East -0.38°
- Andalucia - Sevilla: North 37.4°, West -6.0°, South 37.3°, East -5.9°
- Aragón - Zaragoza: North 41.7°, West -0.8°, South 41.6°, East -0.9°
- Murcia - Murcia: North 38.0°, West -1.1°, South 37.9°, East -1.2°
- País Vasco - Bilbao: North 43.3°, West -2.8°, South 43.2°, East -2.9° 
- Galicia - Vigo: North 42.2°, West -8.6°, South 42.1°, East -8.7°
- Castilla y León - Valladolid: North 41.7°, West -4.6°, South 41.6°, East -4.7°
- Cantabria - Santander: North 43.5°, West -3.7°, South 43.4°, East -3.8°
- La Rioja - Logroño: North 42.5°, West -2.4°, South 42.4°, East -2.5°
- Navarra - Pamplona: North 42.9°, West -1.6°, South 42.8°, East -1.7°
- Castilla la mancha - Albacete: North 39.0°, West -1.8°, South 38.9°, East -1.9°
- Extremadura - Badajoz: North 38.9°, West -6.9°, South 38.8°, East -7.0°

In [22]:
#For each city a file is exported with 4 geographical points
file_list=['Oviedo','Madrid','Barcelona','Valencia','Sevilla','Zaragoza','Malaga','Murcia','Bilbao','Vigo'
           ,'Valladolid']

In [62]:
file_list=['Oviedo','Madrid','Barcelona','Valencia','Sevilla','Zaragoza','Bilbao']

## Data extration

In [63]:
%%time
df_temperatures = []
for file in file_list:
    #Defining paths
    path="../Data/Hourly_Temperature/"+file+".nc"
    
    #Defining empty dataframes (each dataframe is a coordinate point)
    df_point_1=pd.DataFrame()
    df_point_2=pd.DataFrame()
    df_point_3=pd.DataFrame()
    df_point_4=pd.DataFrame() 
    df_time=pd.DataFrame() 
    
    #Defining nc object
    nc=netCDF4.Dataset(path,'r')
    
    #Extracting time
    time=nc.variables['time']
    dtime=netCDF4.num2date(time[:],time.units)
    df_time['Time']=np.array(dtime,dtype=type(dtime))
    
    #Extracting temperatures
    temp_p1=np.array(nc.variables['d2m'][:,0,0],dtype=type(nc.variables['d2m'][:,0,0]))
    temp_p2=np.array(nc.variables['d2m'][:,0,1],dtype=type(nc.variables['d2m'][:,0,1]))
    temp_p3=np.array(nc.variables['d2m'][:,1,0],dtype=type(nc.variables['d2m'][:,1,0]))
    temp_p4=np.array(nc.variables['d2m'][:,1,1],dtype=type(nc.variables['d2m'][:,1,1]))
    
    df_point_1['Temp']=temp_p1
    df_point_2['Temp']=temp_p2
    df_point_3['Temp']=temp_p3
    df_point_4['Temp']=temp_p4
    
    #Concatenation of 4 points     
    df_city=pd.concat([df_point_1,df_point_2,df_point_3,df_point_4])
    
    #Merging time dimension
    df_city=pd.merge(df_city, df_time,left_index=True, right_index=True)
    
    #Including city values
    df_city['City']=file
    
    #Storing each city values
    df_temperatures.append(df_city)

#Concatenating all cities 
df_temperatures = pd.concat(df_temperatures)
df_temperatures.to_csv("../Data/Hourly_Temperature/df_temperatures.csv")

CPU times: user 16.9 s, sys: 21.9 s, total: 38.9 s
Wall time: 1min 27s


## Data transformation: calculating the average of 4 points

In [67]:
#Importing final Dataframe
df_temperatures=pd.read_csv("../Data/Hourly_Temperature/df_temperatures.csv")
df_temperatures

,Unnamed: 0,Temp,Time,City
0,0,269.887412,2015-01-01 00:00:00,Oviedo
1,0,270.258563,2015-01-01 00:00:00,Oviedo
2,0,268.819181,2015-01-01 00:00:00,Oviedo
3,0,269.266227,2015-01-01 00:00:00,Oviedo
4,1,269.308332,2015-01-01 01:00:00,Oviedo
...,...,...,...,...
1739159,62111,278.534944,2022-01-31 23:00:00,Bilbao
1739160,62112,278.601605,2022-02-01 00:00:00,Bilbao
1739161,62112,278.505707,2022-02-01 00:00:00,Bilbao
1739162,62112,278.391098,2022-02-01 00:00:00,Bilbao


In [68]:
#Dropping not needed columns
df_temperatures = df_temperatures.drop('Unnamed: 0', 1)

/Users/malva/miniconda3/envs/final-project/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [70]:
#There are some incorrect values in the dataframe (temperature can not be lower than -283,15k in spain)
#Filtering out those values
df_temperatures=df_temperatures[df_temperatures['Temp']>263.15]

In [71]:
#Calculating the mean of 4 points by City and time
df_temperatures_mean=df_temperatures.groupby(['Time','City'], as_index=False).agg({'Temp':'mean'})
df_temperatures_mean

,Time,City,Temp
0,2015-01-01 00:00:00,Barcelona,276.065316
1,2015-01-01 00:00:00,Bilbao,272.969665
2,2015-01-01 00:00:00,Madrid,269.204407
3,2015-01-01 00:00:00,Oviedo,269.557846
4,2015-01-01 00:00:00,Sevilla,273.404058
...,...,...,...
434719,2022-02-01 00:00:00,Madrid,272.327782
434720,2022-02-01 00:00:00,Oviedo,277.327726
434721,2022-02-01 00:00:00,Sevilla,274.301718
434722,2022-02-01 00:00:00,Valencia,275.516430


In [81]:
# Splitting time
df_temperatures_mean['Time']=pd.to_datetime(df_temperatures_mean['Time'], format="%Y-%m-%d %H:%M:%S")
df_temperatures_mean['Year']=df_temperatures_mean['Time'].dt.year
df_temperatures_mean['Month']=df_temperatures_mean['Time'].dt.month
df_temperatures_mean['Day']=df_temperatures_mean['Time'].dt.day
df_temperatures_mean['Hour']=df_temperatures_mean['Time'].dt.hour
df_temperatures_mean['Date']=pd.to_datetime(df_temperatures_mean['Time'], format="%Y-%m-%d %H:%M:%S")
df_temperatures_mean

,City,Time,Temp,Year,Month,Day,Hour,Date
0,Barcelona,2015-01-01,276.065316,2015,1,1,0,2015-01-01
1,Bilbao,2015-01-01,272.969665,2015,1,1,0,2015-01-01
2,Madrid,2015-01-01,269.204407,2015,1,1,0,2015-01-01
3,Oviedo,2015-01-01,269.557846,2015,1,1,0,2015-01-01
4,Sevilla,2015-01-01,273.404058,2015,1,1,0,2015-01-01
...,...,...,...,...,...,...,...,...
434719,Madrid,2022-02-01,272.327782,2022,2,1,0,2022-02-01
434720,Oviedo,2022-02-01,277.327726,2022,2,1,0,2022-02-01
434721,Sevilla,2022-02-01,274.301718,2022,2,1,0,2022-02-01
434722,Valencia,2022-02-01,275.516430,2022,2,1,0,2022-02-01


In [82]:
df_temperatures_mean = df_temperatures_mean[['City', 'Time','Date', 'Year', 'Month','Day','Hour','Temp']]
df_temperatures_mean

,City,Time,Date,Year,Month,Day,Hour,Temp
0,Barcelona,2015-01-01,2015-01-01,2015,1,1,0,276.065316
1,Bilbao,2015-01-01,2015-01-01,2015,1,1,0,272.969665
2,Madrid,2015-01-01,2015-01-01,2015,1,1,0,269.204407
3,Oviedo,2015-01-01,2015-01-01,2015,1,1,0,269.557846
4,Sevilla,2015-01-01,2015-01-01,2015,1,1,0,273.404058
...,...,...,...,...,...,...,...,...
434719,Madrid,2022-02-01,2022-02-01,2022,2,1,0,272.327782
434720,Oviedo,2022-02-01,2022-02-01,2022,2,1,0,277.327726
434721,Sevilla,2022-02-01,2022-02-01,2022,2,1,0,274.301718
434722,Valencia,2022-02-01,2022-02-01,2022,2,1,0,275.516430


In [83]:
#sending data to csv
df_temperatures_mean.to_csv("../Data/Raw_Data/df_temperatures_mean.csv")